## Agent Skills 实现教程 
### 1. 什么是 Agent Skills？
在 LangChain 的架构中，Skills 和 Tools 有所区别：

- Tools (工具) ：底层功能，如文件系统操作、API 调用、计算器等。
- Skills (技能) ：高层能力，通常是**提示词驱动（Prompt-driven）**的。它们包含详细的指令、参考信息、模板或特定业务逻辑。
核心思想：渐进式披露 (Progressive Disclosure) Skills 的核心优势是减少 Token 消耗。智能体启动时只加载技能的简要描述（Metadata），只有当智能体判断需要该技能时，才会通过工具调用加载完整的技能内容。

### 使用 deepagents 模式

In [ ]:
from deepagents import create_deep_agent
from deepagents.backends.filesystem import FilesystemBackend
from langgraph.checkpoint.memory import MemorySaver

# 1. 设置检查点（实现持久化记忆和审批流）
checkpointer = MemorySaver()

# 2. 创建智能体
agent = create_deep_agent(
    # 指定技能文件所在的根目录
    backend=FilesystemBackend(root_dir="./my_project"),
    # 指定技能存放的具体路径
    skills=["./my_project/skills/"],
    # 配置需要人工干预的操作（可选）
    interrupt_on={
        "write_file": True,
        "edit_file": True
    },
    checkpointer=checkpointer
)

# 3. 调用智能体
result = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": "帮我写一个统计上个月销售额的 SQL。"}
        ]
    },
    config={"configurable": {"thread_id": "session_1"}}
)

### 原生 LangChain 实现 Agent Skills 教程
1. 核心设计思路
1. 技能库 (Skill Registry) ：一个包含技能 ID、简短描述（Metadata）和详细内容（Detailed Prompt）的数据结构。
2. 系统提示词 (System Prompt) ：仅包含技能的 ID 和简短描述，告诉 Agent 它有哪些潜在能力。
3. 加载工具 ( get_skill_details ) ：Agent 用来获取某个技能详细指令的“查手册”工具。

In [ ]:
from typing import Annotated, List, Dict
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
from langgraph.prebuilt import create_react_agent

# --- 第一步：定义技能库 ---
# 实际项目中，这些内容可以存储在数据库或 Markdown 文件中
SKILLS_LIBRARY = {
    "sql_expert": {
        "description": "提供复杂的 SQL 查询编写、优化和数据库架构分析建议。",
        "content": """你现在是 SQL 专家技能模块。
        - 数据库架构：主要表包括 users, transactions, products。
        - 规范：所有查询必须使用标准 SQL，且必须包含对 'deleted_at IS NULL' 的过滤。
        - 性能：优先使用 JOIN 而非子查询。"""
    },
    "data_viz": {
        "description": "提供数据可视化建议，包括图表类型选择和配色方案。",
        "content": """你现在是数据可视化技能模块。
        - 库建议：优先推荐 Plotly 或 Matplotlib。
        - 配色：企业标准色为 #003366 (深蓝) 和 #FFCC00 (金黄)。
        - 逻辑：对于时间序列数据，务必推荐折线图。"""
    }
}

# --- 第二步：定义加载技能的工具 ---
@tool
def get_skill_details(skill_id: str) -> str:
    """当你需要某个特定技能的详细指令、领域知识或操作规范时，调用此工具。
    参数 skill_id 必须是系统提示词中列出的 ID 之一。
    """
    skill = SKILLS_LIBRARY.get(skill_id)
    if skill:
        return f"--- 技能: {skill_id} 的详细内容 ---\n{skill['content']}"
    return f"错误：未找到 ID 为 {skill_id} 的技能。"

# --- 第三步：构建系统提示词 ---
# 仅包含 Metadata，节省 Token
skills_metadata = "\n".join([f"- {sid}: {info['description']}" for sid, info in SKILLS_LIBRARY.items()])

system_prompt = f"""你是一个多功能智能体。你拥有一系列专业技能。
为了保持高效，你初始只知道技能的简介。当你判断需要使用某个技能时，
请务必先调用 `get_skill_details` 工具获取该技能的完整指令。

当前可用技能列表：
{skills_metadata}
"""

# --- 第四步：初始化 Agent ---
model = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [get_skill_details]

# 使用 LangGraph 的预构建 Agent
agent = create_react_agent(model, tools, state_modifier=system_prompt)

# --- 第五步：测试运行 ---
def run_test(query: str):
    print(f"\n用户提问: {query}")
    inputs = {"messages": [HumanMessage(content=query)]}
    for event in agent.stream(inputs, stream_mode="values"):
        message = event["messages"][-1]
        if message.content:
            # 打印 Agent 的思考和回复
            print(f"[{message.type}]: {message.content[:200]}...")

# 运行测试
run_test("我想查询上个月消费最高的 10 个用户，请帮我写个 SQL。")

In [ ]:
from typing import Annotated, List, Dict, TypedDict
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.prebuilt import create_react_agent
from langgraph.graph import StateGraph, END

# --- 1. 定义技能模型 ---
class Skill(BaseModel):
    id: str
    description: str
    content: str

# 模拟技能数据库
SKILLS_REGISTRY: Dict[str, Skill] = {
    "sql_expert": Skill(
        id="sql_expert",
        description="SQL 编写与优化，包含数据库 Schema 知识。",
        content="你现在拥有 SQL 专家技能。规则：1. 必须使用 PostgreSQL 语法；2. 敏感字段需脱敏。"
    ),
    "data_analyzer": Skill(
        id="data_analyzer",
        description="数据趋势分析与统计建议。",
        content="你现在拥有数据分析技能。规则：1. 始终提供同比/环比数据；2. 结果必须包含置信度说明。"
    )
}

# --- 2. 定义状态 (State) ---
# 增加 loaded_skills 来追踪已激活的技能
class AgentState(TypedDict):
    messages: Annotated[List[BaseMessage], "add_messages"]
    loaded_skills: Annotated[List[str], "add_messages"] # 记录已加载的技能 ID

# --- 3. 定义加载工具 ---
@tool
def load_skill(skill_id: str) -> str:
    """激活指定的技能模块。
    当用户的问题涉及 SQL、数据分析等专业领域时，调用此工具以获取详细的操作规范。
    """
    if skill_id in SKILLS_REGISTRY:
        # 在实际逻辑中，这个工具的调用会触发状态更新
        return f"系统通知：技能 '{skill_id}' 已成功激活并注入到你的系统指令中。"
    return f"错误：未找到技能 '{skill_id}'。"

# --- 4. 动态系统提示词构造器 ---
def get_system_prompt(state: AgentState):
    """根据当前状态动态生成 System Prompt"""
    # 基础提示词
    base_prompt = "你是一个智能助手。你可以根据需要通过 load_skill 工具扩展你的能力。\n"
    
    # 注入可用技能清单 (Metadata)
    available_skills = "\n".join([f"- {s.id}: {s.description}" for s in SKILLS_REGISTRY.values()])
    
    # 注入已加载技能的详细内容 (Core Content)
    # 我们从 state 中提取已激活的技能（这里演示从工具调用历史中提取）
    active_instructions = ""
    loaded_ids = []
    
    # 逻辑：遍历消息，寻找 load_skill 的成功调用记录
    for m in state["messages"]:
        if hasattr(m, "tool_calls"):
            for tc in m.tool_calls:
                if tc["name"] == "load_skill":
                    sid = tc["args"].get("skill_id")
                    if sid in SKILLS_REGISTRY and sid not in loaded_ids:
                        loaded_ids.append(sid)
                        active_instructions += f"\n[已激活技能: {sid}]\n{SKILLS_REGISTRY[sid].content}\n"

    full_prompt = f"{base_prompt}\n## 可用技能列表：\n{available_skills}\n"
    if active_instructions:
        full_prompt += f"\n## 当前激活的技能规范：\n{active_instructions}"
    
    return SystemMessage(content=full_prompt)

# --- 5. 创建并运行 Agent ---
model = ChatOpenAI(model="gpt-4o", temperature=0)
tools = [load_skill]

# 使用 state_modifier 动态调整每轮对话的系统提示词
agent = create_react_agent(
    model, 
    tools=tools, 
    state_modifier=get_system_prompt
)

# --- 测试函数 ---
def ask_agent(query: str):
    print(f"\n--- 用户: {query} ---")
    for event in agent.stream({"messages": [("user", query)]}, stream_mode="values"):
        last_msg = event["messages"][-1]
        if last_msg.type == "ai" and last_msg.tool_calls:
            print(f"🤖 [思考] 正在调用工具: {last_msg.tool_calls[0]['name']}...")
        elif last_msg.type == "ai":
            print(f"🤖 [回复]: {last_msg.content}")

# 演示：Agent 会自动判断需要 SQL 技能
ask_agent("帮我写一个查询订单总额的 SQL。")